<a href="https://colab.research.google.com/github/awaiskhan005/DEEP-LEARNING-AND-FORECASTING/blob/main/Reinforcement_learning_for_Crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install 'shimmy>=2.0'  # Compatibility layer for Gymnasium
!pip install gymnasium       # Replace OpenAI Gym with Gymnasium
!pip install stable-baselines3  # Reinstall SB3 to ensure compatibility

In [1]:
pip install gym stable-baselines3 pandas numpy yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [14]:
import gymnasium as gym  # <-- Use gymnasium instead of gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import yfinance as yf


In [15]:
class TradingEnv(gym.Env):
    def __init__(self, data, window_size=30):
        super(TradingEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = self.window_size

        # Action space: 0=hold, 1=buy, 2=sell
        self.action_space = spaces.Discrete(3)

        # Observation space: 6 features (OHLC + Volume + RSI)
        self.observation_space = spaces.Box(  # <-- Fix shape here
            low=-np.inf, high=np.inf,
            shape=(self.window_size, 6),  # Now 6 columns!
            dtype=np.float32
        )

    # ... rest of the code ...

    def _next_observation(self):
        obs = self.data.iloc[self.current_step - self.window_size:self.current_step]
        return obs.values

    def _take_action(self, action):
        current_price = self.data.iloc[self.current_step]['Close']
        if action == 1:  # Buy
            self.position = 1
        elif action == 2:  # Sell
            self.position = -1
        else:  # Hold
            pass

    def step(self, action):
        self._take_action(action)
        self.current_step += 1

        prev_price = self.data.iloc[self.current_step - 1]['Close']
        current_price = self.data.iloc[self.current_step]['Close']
        reward = current_price - prev_price if self.position != 0 else 0

        done = self.current_step >= len(self.data) - 1

        truncated = False  # Gymnasium requires this
        return self._next_observation(), reward, done, truncated, {}

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        self.position = 0
        return self._next_observation(), {}  # Gymnasium expects (obs, info)

In [17]:
import yfinance as yf
import pandas as pd

# Fetch Bitcoin data
data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')

# Calculate RSI
delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))  # <-- Add RSI to DataFrame

# Now select the columns (including RSI)
data = data[['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']]

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[*********************100%***********************]  1 of 1 completed


In [18]:
# Reinitialize and train
env = TradingEnv(data)
env = make_vec_env(lambda: env, n_envs=1)

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10_000)

Using cpu device


ValueError: Expected parameter logits (Tensor of shape (1, 3)) of distribution Categorical(logits: torch.Size([1, 3])) to satisfy the constraint IndependentConstraint(Real(), 1), but found invalid values:
tensor([[nan, nan, nan]])

In [ ]:
obs = env.reset()
done = False
total_reward = 0

while not done:
    action, _ = model.predict(obs)
    obs, reward, done, _ = env.step(action)
    total_reward += reward

print(f"Total simulated profit: {total_reward}")

In [20]:

import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import yfinance as yf
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# ======================
# 1. Trading Environment
# ======================
class TradingEnv(gym.Env):
    def __init__(self, data, window_size=30):
        super(TradingEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = self.window_size
        self.position = 0  # Current holding position

        # Action space: 0=hold, 1=buy, 2=sell
        self.action_space = spaces.Discrete(3)

        # Observation space: 6 features (OHLC + Volume + RSI)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(self.window_size, 6),
            dtype=np.float32
        )

    def _next_observation(self):
        # Get current window of data
        return self.data.iloc[
            self.current_step - self.window_size:self.current_step
        ].values

    def _take_action(self, action):
        # Update position based on action
        if action == 1:  # Buy
            self.position = 1
        elif action == 2:  # Sell
            self.position = -1
        # Hold does nothing

    def step(self, action):
        self._take_action(action)
        self.current_step += 1

        # Calculate reward (price difference)
        prev_price = self.data.iloc[self.current_step - 1]['Close']
        current_price = self.data.iloc[self.current_step]['Close']
        reward = current_price - prev_price if self.position != 0 else 0

        # Check termination
        done = self.current_step >= len(self.data) - 1
        truncated = False  # Gymnasium requirement

        return self._next_observation(), reward, done, truncated, {}

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        self.position = 0
        return self._next_observation(), {}  # (obs, info)

# ====================
# 2. Data Preparation
# ====================
# Fetch Bitcoin data
data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')

# Calculate RSI
delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))

# Select columns and clean NaN values
data = data[['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].dropna()

# ====================
# 3. Train RL Agent
# ====================
# Create environment
env = TradingEnv(data)
env = make_vec_env(lambda: env, n_envs=1)

# Initialize and train PPO agent
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10_000)

# Save model
model.save("crypto_trading_agent")

# ====================
# Reset the vectorized environment
obs = env.reset()
done = [False]
total_reward = 0

while not done[0]:
    action, _ = model.predict(obs)
    obs, rewards, done, _ = env.step(action)
    total_reward += rewards[0]

print(f"Total simulated profit: ${total_reward:.2f}")

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/monitor.py:95: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  self.rewards.append(float(reward))
/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:59: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(  # type: ignore[assignment]


Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.05e+03 |
|    ep_rew_mean     | 6.33e+03 |
| time/              |          |
|    fps             | 664      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.05e+03    |
|    ep_rew_mean          | 6.3e+03     |
| time/                   |             |
|    fps                  | 611         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008479201 |
|    clip_fraction        | 0.0451      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -8.23e-06   |
|    learning

In [26]:


# Import libraries
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import yfinance as yf
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from sklearn.preprocessing import MinMaxScaler

# ======================
# 1. Data Preparation
# ======================
# Fetch and preprocess data
data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')

# Calculate RSI
delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))
data = data[['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].dropna()

# Scale data for LSTM
close_scaler = MinMaxScaler()
data['Close_scaled'] = close_scaler.fit_transform(data[['Close']])

# Create sequences for LSTM
sequence_length = 30
X, y = [], []
for i in range(len(data)-sequence_length-1):
    X.append(data['Close_scaled'].values[i:i+sequence_length])
    y.append(data['Close_scaled'].values[i+sequence_length+1])  # Predict next day
X, y = np.array(X), np.array(y)

# Split data (80% train, 20% test)
split = int(0.8*len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# ======================
# 2. LSTM Price Predictor
# ======================
class PricePredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(50, 25),
            nn.ReLU(),
            nn.Linear(25, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(-1)  # Add feature dimension
        out, _ = self.lstm(x)
        return self.fc(out[:, -1])

# Initialize model
predictor = PricePredictor()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(predictor.parameters(), lr=0.001)

# Convert to tensors
train_data = torch.utils.data.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# Train LSTM
epochs = 50
for epoch in range(epochs):
    for batch_x, batch_y in train_loader:
        outputs = predictor(batch_x)
        loss = criterion(outputs.squeeze(), batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Save predictor
torch.save(predictor.state_dict(), "lstm_predictor.pth")

# ======================
# 3. Hybrid RL Environment
# ======================
class HybridTradingEnv(gym.Env):
    def __init__(self, data, predictor):
        super().__init__()
        self.data = data
        self.predictor = predictor
        self.window_size = 30
        self.current_step = self.window_size
        self.position = 0
        self.transaction_cost = 0.001  # 0.1% per trade

        # Action space: 0=hold, 1=buy, 2=sell
        self.action_space = spaces.Discrete(3)

        # Observation space: OHLCV + RSI + Predicted_Price
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(self.window_size, 7),
            dtype=np.float32
        )

```python
# Import libraries
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import yfinance as yf
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from sklearn.preprocessing import MinMaxScaler

# ======================
# 1. Data Preparation
# ======================
# Fetch and preprocess data
data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')

# Calculate RSI
delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))
data = data[['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].dropna()

# Scale data for LSTM
close_scaler = MinMaxScaler()
data['Close_scaled'] = close_scaler.fit_transform(data[['Close']])

# Create sequences for LSTM
sequence_length = 30
X, y = [], []
for i in range(len(data)-sequence_length-1):
    X.append(data['Close_scaled'].values[i:i+sequence_length])
    y.append(data['Close_scaled'].values[i+sequence_length+1])  # Predict next day
X, y = np.array(X), np.array(y)

# Split data (80% train, 20% test)
split = int(0.8*len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# ======================
# 2. LSTM Price Predictor
# ======================
class PricePredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(50, 25),
            nn.ReLU(),
            nn.Linear(25, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(-1)  # Add feature dimension
        out, _ = self.lstm(x)
        return self.fc(out[:, -1])

# Initialize model
predictor = PricePredictor()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(predictor.parameters(), lr=0.001)

# Convert to tensors
train_data = torch.utils.data.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# Train LSTM
epochs = 50
for epoch in range(epochs):
    for batch_x, batch_y in train_loader:
        outputs = predictor(batch_x)
        loss = criterion(outputs.squeeze(), batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Save predictor
torch.save(predictor.state_dict(), "lstm_predictor.pth")

# ======================
# 3. Hybrid RL Environment
# ======================
class HybridTradingEnv(gym.Env):
    def __init__(self, data, predictor):
        super().__init__()
        self.data = data
        self.predictor = predictor
        self.window_size = 30
        self.current_step = self.window_size
        self.position = 0
        self.transaction_cost = 0.001  # 0.1% per trade
        self.balance = 0 # initialize balance

        # Action space: 0=hold, 1=buy, 2=sell
        self.action_space = spaces.Discrete(3)

        # Observation space: OHLCV + RSI + Predicted_Price
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(self.window_size, 7),
            dtype=np.float32
        )

    def _get_prediction(self, window):
        with torch.no_grad():
            return self.predictor(torch.FloatTensor(window)).item()

    def _next_observation(self):
        window = self.data.iloc[
            self.current_step - self.window_size:self.current_step
        ][['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].values

        # Get prediction for next day
        prediction = self._get_prediction(
            self.data['Close_scaled'].values[
                self.current_step - self.window_size:self.current_step
            ]
        )

        # Add prediction to observation
        return np.concatenate([window, np.full((self.window_size, 1), prediction)], axis=1)

    def _take_action(self, action):
        current_price = self.data.iloc[self.current_step]['Close']
        if action == 1 and self.position <= 0:  # Buy
            self.position = 1
            self.balance -= current_price * (1 + self.transaction_cost) # Consider transaction cost on buy
        elif action == 2 and self.position >= 0:  # Sell
            self.position = -1
            self.balance += current_price * (1 - self.transaction_cost) # Consider transaction cost on sell


    def step(self, action):
        self._take_action(action)
        self.current_step += 1

        prev_price = self.data.iloc[self.current_step - 1]['Close']
        current_price = self.data.iloc[self.current_step]['Close']

        # Risk-adjusted reward
        price_change = current_price - prev_price
        reward = price_change * self.position  - abs(price_change) * self.transaction_cost

        done = self.current_step >= len(self.data) - 1
        truncated = False

        return self._next_observation(), reward, done, truncated, {}

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        self.position = 0
        self.balance = 0
        return self._next_observation(), {}

# ======================
# 4. Train RL Agent
# ======================
# Load predictor
predictor.load_state_dict(torch.load("lstm_predictor.pth"))
predictor.eval()

# Create environment
env = HybridTradingEnv(data, predictor)
env = make_vec_env(lambda: env, n_envs=1)

# Initialize and train PPO agent
model = PPO("MlpPolicy", env, verbose=1,
            policy_kwargs=dict(net_arch=dict(pi=[256, 256], vf=[256, 256])))
model.learn(total_timesteps=50_000)

# Save model
model.save("hybrid_trading_agent")

# ======================
# 5. Test & Predict
# ======================
def predict_next_day():
    # Get latest data
    latest_data = data['Close_scaled'].values[-sequence_length:]

    # Predict next day
    with torch.no_grad():
        prediction = predictor(torch.FloatTensor(latest_data))

    # Inverse transform
    return close_scaler.inverse_transform(prediction.detach().numpy().reshape(1,-1))[0][0]

# Get predictions
next_day_price = predict_next_day()
print(f"Predicted next day closing price: ${next_day_price:.2f}")

# Test agent
obs, info = env.reset()
done = False
total_reward = 0

while not done:
```python
# Import libraries
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import yfinance as yf
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from sklearn.preprocessing import MinMaxScaler

# ======================
# 1. Data Preparation
# ======================
# Fetch and preprocess data
data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')

# Calculate RSI
delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))
data = data[['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].dropna()

# Scale data for LSTM
close_scaler = MinMaxScaler()
data['Close_scaled'] = close_scaler.fit_transform(data[['Close']])

# Create sequences for LSTM
sequence_length = 30
X, y = [], []
for i in range(len(data)-sequence_length-1):
    X.append(data['Close_scaled'].values[i:i+sequence_length])
    y.append(data['Close_scaled'].values[i+sequence_length+1])  # Predict next day
X, y = np.array(X), np.array(y)

# Split data (80% train, 20% test)
split = int(0.8*len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# ======================
# 2. LSTM Price Predictor
# ======================
class PricePredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(50, 25),
            nn.ReLU(),
            nn.Linear(25, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(-1)  # Add feature dimension
        out, _ = self.lstm(x)
        return self.fc(out[:, -1])

# Initialize model
predictor = PricePredictor()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(predictor.parameters(), lr=0.001)

# Convert to tensors
train_data = torch.utils.data.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# Train LSTM
epochs = 50
for epoch in range(epochs):
    for batch_x, batch_y in train_loader:
        outputs = predictor(batch_x)
        loss = criterion(outputs.squeeze(), batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Save predictor
torch.save(predictor.state_dict(), "lstm_predictor.pth")

# ======================
# 3. Hybrid RL Environment
# ======================
class HybridTradingEnv(gym.Env):
    def __init__(self, data, predictor):
        super().__init__()
        self.data = data
        self.predictor = predictor
        self.window_size = 30
        self.current_step = self.window_size
        self.position = 0
        self.transaction_cost = 0.001  # 0.1% per trade
        self.balance = 0 # initialize balance

        # Action space: 0=hold, 1=buy, 2=sell
        self.action_space = spaces.Discrete(3)

        # Observation space: OHLCV + RSI + Predicted_Price
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(self.window_size, 7),
            dtype=np.float32
        )

    def _get_prediction(self, window):
        with torch.no_grad():
            return self.predictor(torch.FloatTensor(window)).item()

    def _next_observation(self):
        window = self.data.iloc[
            self.current_step - self.window_size:self.current_step
        ][['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].values

        # Get prediction for next day
        prediction = self._get_prediction(
            self.data['Close_scaled'].values[
                self.current_step - self.window_size:self.current_step
            ]
        )

        # Add prediction to observation
        return np.concatenate([window, np.full((self.window_size, 1), prediction)], axis=1)

    def _take_action(self, action):
        current_price = self.data.iloc[self.current_step]['Close']
        if action == 1 and self.position <= 0:  # Buy
            self.position = 1
            self.balance -= current_price * (1 + self.transaction_cost) # Consider transaction cost on buy
        elif action == 2 and self.position >= 0:  # Sell
            self.position = -1
            self.balance += current_price * (1 - self.transaction_cost) # Consider transaction cost on sell


    def step(self, action):
        self._take_action(action)
        self.current_step += 1

        prev_price = self.data.iloc[self.current_step - 1]['Close']
        current_price = self.data.iloc[self.current_step]['Close']

        # Risk-adjusted reward
        price_change = current_price - prev_price
        reward = price_change * self.position  - abs(price_change) * self.transaction_cost

        done = self.current_step >= len(self.data) - 1
        truncated = False

        return self._next_observation(), reward, done, truncated, {}

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        self.position = 0
        self.balance = 0
        return self._next_observation(), {}

# ======================
# 4. Train RL Agent
# ======================
# Load predictor
predictor.load_state_dict(torch.load("lstm_predictor.pth"))
predictor.eval()

# Create environment
env = HybridTradingEnv(data, predictor)
env = make_vec_env(lambda: env, n_envs=1)

# Initialize and train PPO agent
model = PPO("MlpPolicy", env, verbose=1,
            policy_kwargs=dict(net_arch=dict(pi=[256, 256], vf=[256, 256])))
model.learn(total_timesteps=50_000)

# Save model
model.save("hybrid_trading_agent")

# ======================
# 5. Test & Predict
# ======================
def predict_next_day():
    # Get latest data
    latest_data = data['Close_scaled'].values[-sequence_length:]

    # Predict next day
    with torch.no_grad():
        prediction = predictor(torch.FloatTensor(latest_data))

    # Inverse transform
    return close_scaler.inverse_transform(prediction.detach().numpy().reshape(1,-1))[0][0]

# Get predictions
next_day_price = predict_next_day()
print(f"Predicted next day closing price: ${next_day_price:.2f}")

# Test agent
obs, info = env.reset()
done = False
total_reward = 0

while not done:
```python
# Import libraries
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import yfinance as yf
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from sklearn.preprocessing import MinMaxScaler

# ======================
# 1. Data Preparation
# ======================
# Fetch and preprocess data
data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')

# Calculate RSI
delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))
data = data[['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].dropna()

# Scale data for LSTM
close_scaler = MinMaxScaler()
data['Close_scaled'] = close_scaler.fit_transform(data[['Close']])

# Create sequences for LSTM
sequence_length = 30
X, y = [], []
for i in range(len(data)-sequence_length-1):
    X.append(data['Close_scaled'].values[i:i+sequence_length])
    y.append(data['Close_scaled'].values[i+sequence_length+1])  # Predict next day
X, y = np.array(X), np.array(y)

# Split data (80% train, 20% test)
split = int(0.8*len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# ======================
# 2. LSTM Price Predictor
# ======================
class PricePredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(50, 25),
            nn.ReLU(),
            nn.Linear(25, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(-1)  # Add feature dimension
        out, _ = self.lstm(x)
        return self.fc(out[:, -1])

# Initialize model
predictor = PricePredictor()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(predictor.parameters(), lr=0.001)

# Convert to tensors
train_data = torch.utils.data.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# Train LSTM
epochs = 50
for epoch in range(epochs):
    for batch_x, batch_y in train_loader:
        outputs = predictor(batch_x)
        loss = criterion(outputs.squeeze(), batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Save predictor
torch.save(predictor.state_dict(), "lstm_predictor.pth")

# ======================
# 3. Hybrid RL Environment
# ======================
class HybridTradingEnv(gym.Env):
    def __init__(self, data, predictor):
        super().__init__()
        self.data = data
        self.predictor = predictor
        self.window_size = 30
        self.current_step = self.window_size
        self.position = 0
        self.transaction_cost = 0.001  # 0.1% per trade
        self.balance = 0 # initialize balance

        # Action space: 0=hold, 1=buy, 2=sell
        self.action_space = spaces.Discrete(3)

        # Observation space: OHLCV + RSI + Predicted_Price
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(self.window_size, 7),
            dtype=np.float32
        )

    def _get_prediction(self, window):
        with torch.no_grad():
            return self.predictor(torch.FloatTensor(window)).item()

    def _next_observation(self):
        window = self.data.iloc[
            self.current_step - self.window_size:self.current_step
        ][['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].values

        # Get prediction for next day
        prediction = self._get_prediction(
            self.data['Close_scaled'].values[
                self.current_step - self.window_size:self.current_step
            ]
        )

        # Add prediction to observation
        return np.concatenate([window, np.full((self.window_size, 1), prediction)], axis=1)

    def _take_action(self, action):
        current_price = self.data.iloc[self.current_step]['Close']
        if action == 1 and self.position <= 0:  # Buy
            self.position = 1
            self.balance -= current_price * (1 + self.transaction_cost) # Consider transaction cost on buy
        elif action == 2 and self.position >= 0:  # Sell
            self.position = -1
            self.balance += current_price * (1 - self.transaction_cost) # Consider transaction cost on sell


    def step(self, action):
        self._take_action(action)
        self.current_step += 1

        prev_price = self.data.iloc[self.current_step - 1]['Close']
        current_price = self.data.iloc[self.current_step]['Close']

        # Risk-adjusted reward
        price_change = current_price - prev_price
        reward = price_change * self.position  - abs(price_change) * self.transaction_cost

        done = self.current_step >= len(self.data) - 1
        truncated = False

        return self._next_observation(), reward, done, truncated, {}

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        self.position = 0
        self.balance = 0
        return self._next_observation(), {}

# ======================
# 4. Train RL Agent
# ======================
# Load predictor
predictor.load_state_dict(torch.load("lstm_predictor.pth"))
predictor.eval()

# Create environment
env = HybridTradingEnv(data, predictor)
env = make_vec_env(lambda: env, n_envs=1)

# Initialize and train PPO agent
model = PPO("MlpPolicy", env, verbose=1,
            policy_kwargs=dict(net_arch=dict(pi=[256, 256], vf=[256, 256])))
model.learn(total_timesteps=50_000)

# Save model
model.save("hybrid_trading_agent")

# ======================
# 5. Test & Predict
# ======================
def predict_next_day():
    # Get latest data
    latest_data = data['Close_scaled'].values[-sequence_length:]

    # Predict next day
    with torch.no_grad():
        prediction = predictor(torch.FloatTensor(latest_data))

    # Inverse transform
    return close_scaler.inverse_transform(prediction.detach().numpy().reshape(1,-1))[0][0]

# Get predictions
next_day_price = predict_next_day()
print(f"Predicted next day closing price: ${next_day_price:.2f}")

# Test agent
obs, info = env.reset()
done = False
total_reward = 0

while not done:
```python
# Import libraries
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import yfinance as yf
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from sklearn.preprocessing import MinMaxScaler

# ======================
# 1. Data Preparation
# ======================
# Fetch and preprocess data
data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')

# Calculate RSI
delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))
data = data[['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].dropna()

# Scale data for LSTM
close_scaler = MinMaxScaler()
data['Close_scaled'] = close_scaler.fit_transform(data[['Close']])

# Create sequences for LSTM
sequence_length = 30
X, y = [], []
for i in range(len(data)-sequence_length-1):
    X.append(data['Close_scaled'].values[i:i+sequence_length])
    y.append(data['Close_scaled'].values[i+sequence_length+1])  # Predict next day
X, y = np.array(X), np.array(y)

# Split data (80% train, 20% test)
split = int(0.8*len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# ======================
# 2. LSTM Price Predictor
# ======================
class PricePredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(50, 25),
            nn.ReLU(),
            nn.Linear(25, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(-1)  # Add feature dimension
        out, _ = self.lstm(x)
        return self.fc(out[:, -1])

# Initialize model
predictor = PricePredictor()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(predictor.parameters(), lr=0.001)

# Convert to tensors
train_data = torch.utils.data.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# Train LSTM
epochs = 50
for epoch in range(epochs):
    for batch_x, batch_y in train_loader:
        outputs = predictor(batch_x)
        loss = criterion(outputs.squeeze(), batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Save predictor
torch.save(predictor.state_dict(), "lstm_predictor.pth")

# ======================
# 3. Hybrid RL Environment
# ======================
class HybridTradingEnv(gym.Env):
    def __init__(self, data, predictor):
        super().__init__()
        self.data = data
        self.predictor = predictor
        self.window_size = 30
        self.current_step = self.window_size
        self.position = 0
        self.transaction_cost = 0.001  # 0.1% per trade
        self.balance = 0 # initialize balance

        # Action space: 0=hold, 1=buy, 2=sell
        self.action_space = spaces.Discrete(3)

        # Observation space: OHLCV + RSI + Predicted_Price
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(self.window_size, 7),
            dtype=np.float32
        )

    def _get_prediction(self, window):
        with torch.no_grad():
            return self.predictor(torch.FloatTensor(window)).item()

    def _next_observation(self):
        window = self.data.iloc[
            self.current_step - self.window_size:self.current_step
        ][['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].values

        # Get prediction for next day
        prediction = self._get_prediction(
            self.data['Close_scaled'].values[
                self.current_step - self.window_size:self.current_step
            ]
        )

        # Add prediction to observation
        return np.concatenate([window, np.full((self.window_size, 1), prediction)], axis=1)

    def _take_action(self, action):
        current_price = self.data.iloc[self.current_step]['Close']
        if action == 1 and self.position <= 0:  # Buy
            self.position = 1
            self.balance -= current_price * (1 + self.transaction_cost) # Consider transaction cost on buy
        elif action == 2 and self.position >= 0:  # Sell
            self.position = -1
            self.balance += current_price * (1 - self.transaction_cost) # Consider transaction cost on sell


    def step(self, action):
        self._take_action(action)
        self.current_step += 1

        prev_price = self.data.iloc[self.current_step - 1]['Close']
        current_price = self.data.iloc[self.current_step]['Close']

        # Risk-adjusted reward
        price_change = current_price - prev_price
        reward = price_change * self.position  - abs(price_change) * self.transaction_cost

        done = self.current_step >= len(self.data) - 1
        truncated = False

        return self._next_observation(), reward, done, truncated, {}

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        self.position = 0
        self.balance = 0
        return self._next_observation(), {}

# ======================
# 4. Train RL Agent
# ======================
# Load predictor
predictor.load_state_dict(torch.load("lstm_predictor.pth"))
predictor.eval()

# Create environment
env = HybridTradingEnv(data, predictor)
env = make_vec_env(lambda: env, n_envs=1)

# Initialize and train PPO agent
model = PPO("MlpPolicy", env, verbose=1,
            policy_kwargs=dict(net_arch=dict(pi=[256, 256], vf=[256, 256])))
model.learn(total_timesteps=50_000)

# Save model
model.save("hybrid_trading_agent")

# ======================
# 5. Test & Predict
# ======================
def predict_next_day():
    # Get latest data
    latest_data = data['Close_scaled'].values[-sequence_length:]

    # Predict next day
    with torch.no_grad():
        prediction = predictor(torch.FloatTensor(latest_data))

    # Inverse transform
    return close_scaler.inverse_transform(prediction.detach().numpy().reshape(1,-1))[0][0]

# Get predictions
next_day_price = predict_next_day()
print(f"Predicted next day closing price: ${next_day_price:.2f}")

# Test agent
obs, info = env.reset()
done = False
total_reward = 0

while not done:
```python
# Import libraries
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import yfinance as yf
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from sklearn.preprocessing import MinMaxScaler

# ======================
# 1. Data Preparation
# ======================
# Fetch and preprocess data
data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')

# Calculate RSI
delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))
data = data[['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].dropna()

# Scale data for LSTM
close_scaler = MinMaxScaler()
data['Close_scaled'] = close_scaler.fit_transform(data[['Close']])

# Create sequences for LSTM
sequence_length = 30
X, y = [], []
for i in range(len(data)-sequence_length-1):
    X.append(data['Close_scaled'].values[i:i+sequence_length])
    y.append(data['Close_scaled'].values[i+sequence_length+1])  # Predict next day
X, y = np.array(X), np.array(y)

# Split data (80% train, 20% test)
split = int(0.8*len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# ======================
# 2. LSTM Price Predictor
# ======================
class PricePredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(50, 25),
            nn.ReLU(),
            nn.Linear(25, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(-1)  # Add feature dimension
        out, _ = self.lstm(x)
        return self.fc(out[:, -1])

# Initialize model
predictor = PricePredictor()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(predictor.parameters(), lr=0.001)

# Convert to tensors
train_data = torch.utils.data.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# Train LSTM
epochs = 50
for epoch in range(epochs):
    for batch_x, batch_y in train_loader:
        outputs = predictor(batch_x)
        loss = criterion(outputs.squeeze(), batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Save predictor
torch.save(predictor.state_dict(), "lstm_predictor.pth")

# ======================
# 3. Hybrid RL Environment
# ======================
class HybridTradingEnv(gym.Env):
    def __init__(self, data, predictor):
        super().__init__()
        self.data = data
        self.predictor = predictor
        self.window_size = 30
        self.current_step = self.window_size
        self.position = 0
        self.transaction_cost = 0.001  # 0.1% per trade
        self.balance = 0 # initialize balance

        # Action space: 0=hold, 1=buy, 2=sell
        self.action_space = spaces.Discrete(3)

        # Observation space: OHLCV + RSI + Predicted_Price
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(self.window_size, 7),
            dtype=np.float32
        )

    def _get_prediction(self, window):
        with torch.no_grad():
            return self.predictor(torch.FloatTensor(window)).item()

    def _next_observation(self):
        window = self.data.iloc[
            self.current_step - self.window_size:self.current_step
        ][['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].values

        # Get prediction for next day
        prediction = self._get_prediction(
            self.data['Close_scaled'].values[
                self.current_step - self.window_size:self.current_step
            ]
        )

        # Add prediction to observation
        return np.concatenate([window, np.full((self.window_size, 1), prediction)], axis=1)

    def _take_action(self, action):
        current_price = self.data.iloc[self.current_step]['Close']
        if action == 1 and self.position <= 0:  # Buy
            self.position = 1
            self.balance -= current_price * (1 + self.transaction_cost) # Consider transaction cost on buy
        elif action == 2 and self.position >= 0:  # Sell
            self.position = -1
            self.balance += current_price * (1 - self.transaction_cost) # Consider transaction cost on sell


    def step(self, action):
        self._take_action(action)
        self.current_step += 1

        prev_price = self.data.iloc[self.current_step - 1]['Close']
        current_price = self.data.iloc[self.current_step]['Close']

        # Risk-adjusted reward
        price_change = current_price - prev_price
        reward = price_change * self.position  - abs(price_change) * self.transaction_cost

        done = self.current_step >= len(self.data) - 1
        truncated = False

        return self._next_observation(), reward, done, truncated, {}

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        self.position = 0
        self.balance = 0
        return self._next_observation(), {}

# ======================
# 4. Train RL Agent
# ======================
# Load predictor
predictor.load_state_dict(torch.load("lstm_predictor.pth"))
predictor.eval()

# Create environment
env = HybridTradingEnv(data, predictor)
env = make_vec_env(lambda: env, n_envs=1)

# Initialize and train PPO agent
model = PPO("MlpPolicy", env, verbose=1,
            policy_kwargs=dict(net_arch=dict(pi=[256, 256], vf=[256, 256])))
model.learn(total_timesteps=50_000)

# Save model
model.save("hybrid_trading_agent")

# ======================
# 5. Test & Predict
# ======================
def predict_next_day():
    # Get latest data
    latest_data = data['Close_scaled'].values[-sequence_length:]

    # Predict next day
    with torch.no_grad():
        prediction = predictor(torch.FloatTensor(latest_data))

    # Inverse transform
    return close_scaler.inverse_transform(prediction.detach().numpy().reshape(1,-1))[0][0]

# Get predictions
next_day_price = predict_next_day()
print(f"Predicted next day closing price: ${next_day_price:.2f}")

# Test agent
obs, info = env.reset()
done = False
total_reward = 0

while not done:
    action, _ = model.predict(obs)
    obs, reward, done, truncated, info  = env.step(action[0])
    total_reward += reward
print(f"Total simulated profit with hybrid agent_)
        window = self.data.iloc[
            self.current_step - self.window_size:self.current_step
        ][['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].values

        # Get prediction for next day
        prediction = self._get_prediction(
            self.data['Close_scaled'].values[
                self.current_step - self.window_size:self.current_step
            ]
        )

        # Add prediction to observation
        return np.concatenate([window, np.full((self.window_size, 1), prediction)], axis=1)

    def _take_action(self, action):
        current_price = self.data.iloc[self.current_step]['Close']
        if action == 1 and self.position <= 0:  # Buy
            self.position = 1
            self.balance -= current_price * self.transaction_cost
        elif action == 2 and self.position >= 0:  # Sell
            self.position = -1
            self.balance -= current_price * self.transaction_cost

    def step(self, action):
        self._take_action(action)
        self.current_step += 1

        prev_price = self.data.iloc[self.current_step - 1]['Close']
        current_price = self.data.iloc[self.current_step]['Close']

        # Risk-adjusted reward
        price_change = current_price - prev_price
        reward = price_change * self.position - abs(price_change) * self.transaction_cost

        done = self.current_step >= len(self.data) - 1
        truncated = False

        return self._next_observation(), reward, done, truncated, {}

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        self.position = 0
        self.balance = 0
        return self._next_observation(), {}

# ======================
# 4. Train RL Agent
# ======================
# Load predictor
predictor.load_state_dict(torch.load("lstm_predictor.pth"))
predictor.eval()

# Create environment
env = HybridTradingEnv(data, predictor)
env = make_vec_env(lambda: env, n_envs=1)

# Initialize and train PPO agent
model = PPO("MlpPolicy", env, verbose=1,
            policy_kwargs=dict(net_arch=dict(pi=[256, 256], vf=[256, 256])))
model.learn(total_timesteps=50_000)

# Save model
model.save("hybrid_trading_agent")

# ======================
# 5. Test & Predict
# ======================
def predict_next_day():
    # Get latest data
    latest_data = data['Close_scaled'].values[-sequence_length:]

    # Predict next day
    with torch.no_grad():
        prediction = predictor(torch.FloatTensor(latest_data).item()

    # Inverse transform
    return close_scaler.inverse_transform([[prediction]])[0][0]

# Get predictions
next_day_price = predict_next_day()
print(f"Predicted next day closing price: ${next_day_price:.2f}")

# Test agent
obs = env.reset()
done = [False]
total_reward = 0

while not done[0]:
    action, _ = model.predict(obs)
    obs, rewards, done, _ = env.step(action)
    total_reward += rewards[0]

print(f"Total simulated profit with hybrid agent: ${total_reward:.2f}")

SyntaxError: unterminated string literal (detected at line 1138) (<ipython-input-26-89701fc75336>, line 1138)